In [1]:
import os

import numpy as np
import re

from tensorflow.keras.layers import Dense, GRU, Input, Dropout, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Позитивний текст
texts_true = [
    "Я обожнюю подорожувати! Відкриття нових місць та культур приносить мені велику радість.",
    "Нещодавно я відвідав(ла) Італію і був(ла) вражений(а) красою Риму та Флоренції.",
    "Планую поїздку до Японії, щоб побачити гору Фудзі та інші пам'ятки.",
]

# Негативний текст
texts_false = [
    "Подорожі - це занадто дорого, і я не можу собі їх дозволити.",
    "Я не люблю літати, тому я уникаю подорожей за кордон.",
    "Я боюся їздити в незнайомі місця, тому волію залишатися вдома.",
]

texts = texts_true + texts_false
count_true = len(texts_true)
count_false = len(texts_false)
total_lines = count_true + count_false
print(count_true, count_false, total_lines)


maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(texts)

dist = list(tokenizer.word_counts.items())
print(dist[:10])
print(texts[0][:100])


max_text_len = 10
data = tokenizer.texts_to_sequences(texts)
data_pad = pad_sequences(data, maxlen=max_text_len)
print(data_pad)

print( list(tokenizer.word_index.items()) )


X = data_pad
Y = np.array([[1, 0]]*count_true + [[0, 1]]*count_false)
print(X.shape, Y.shape)

indeces = np.random.choice(X.shape[0], size=X.shape[0], replace=False)
X = X[indeces]
Y = Y[indeces]


model = Sequential()
model.add(Embedding(maxWordsCount, 256, input_length = max_text_len))
model.add(GRU(128, return_sequences=True))
model.add(GRU(128, return_sequences=True))
model.add(GRU(64))
model.add(Dense(2, activation='sigmoid'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=RMSprop(lr=0.00005))

history = model.fit(X, Y, batch_size=32, epochs=50)

reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

def sequence_to_text(list_of_indices):
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)

3 3 6
[('я', 6), ('обожнюю', 1), ('подорожувати', 1), ('відкриття', 1), ('нових', 1), ('місць', 1), ('та', 3), ('культур', 1), ('приносить', 1), ('мені', 1)]
Я обожнюю подорожувати! Відкриття нових місць та культур приносить мені велику радість.
[[ 8  9 10 11  2 12 13 14 15 16]
 [19  4 20  3 21 22 23 24  2 25]
 [27 28 29 30 31 32 33  2 34 35]
 [37 38 39  4  1  5 40 41 42 43]
 [ 1  5 44 45  6  1 46 47 48 49]
 [ 1 50 51 52 53 54  6 55 56 57]]
[('я', 1), ('та', 2), ('ла', 3), ('і', 4), ('не', 5), ('тому', 6), ('обожнюю', 7), ('подорожувати', 8), ('відкриття', 9), ('нових', 10), ('місць', 11), ('культур', 12), ('приносить', 13), ('мені', 14), ('велику', 15), ('радість', 16), ('нещодавно', 17), ('відвідав', 18), ('італію', 19), ('був', 20), ('вражений', 21), ('а', 22), ('красою', 23), ('риму', 24), ('флоренції', 25), ('планую', 26), ('поїздку', 27), ('до', 28), ('японії', 29), ('щоб', 30), ('побачити', 31), ('гору', 32), ('фудзі', 33), ('інші', 34), ("пам'ятки", 35), ('подорожі', 36), ('це'

Epoch 1/50
1/1 [==============================] - 5s 5s/step - loss: 0.6952 - accuracy: 0.5000
Epoch 2/50
1/1 [==============================] - 0s 19ms/step - loss: 0.6582 - accuracy: 1.0000
Epoch 3/50
1/1 [==============================] - 0s 16ms/step - loss: 0.6043 - accuracy: 0.8333
Epoch 4/50
1/1 [==============================] - 0s 15ms/step - loss: 0.4738 - accuracy: 1.0000
Epoch 5/50
1/1 [==============================] - 0s 17ms/step - loss: 0.2367 - accuracy: 1.0000
Epoch 6/50
1/1 [==============================] - 0s 15ms/step - loss: 0.0493 - accuracy: 1.0000
Epoch 7/50
1/1 [==============================] - 0s 15ms/step - loss: 0.0128 - accuracy: 1.0000
Epoch 8/50
1/1 [==============================] - 0s 14ms/step - loss: 0.0072 - accuracy: 1.0000
Epoch 9/50
1/1 [==============================] - 0s 18ms/step - loss: 0.0049 - accuracy: 1.0000
Epoch 10/50
1/1 [==============================] - 0s 18ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 11/50
1/1 [==============

In [2]:
t = "Мені приносить велику радість відвідування ноних місць.".lower()
data = tokenizer.texts_to_sequences([t])
data_pad = pad_sequences(data, maxlen=max_text_len)
print( sequence_to_text(data[0]) )

res = model.predict(data_pad)


['мені', 'приносить', 'велику', 'радість', 'місць']
1/1 [==============================] - 1s 922ms/step


In [3]:
# Визначення результату
if np.argmax(res) == 0:
    print("Результат. Текст позитивний.")
else:
    print("Результат. Текст негативний.")

Результат. Текст позитивний.


In [4]:
t1 = "Я боюсь літати, тому сиджу вдома.".lower()
data1 = tokenizer.texts_to_sequences([t1])
data1_pad = pad_sequences(data1, maxlen=max_text_len)
print( sequence_to_text(data1[0]) )

res1 = model.predict(data1_pad)


['я', 'літати', 'тому', 'вдома']
1/1 [==============================] - 0s 19ms/step


In [5]:
# Визначення результату
if np.argmax(res) == 0:
    print("Результат. Текст негативний.")
else:
    print("Результат. Текст позитивний.")

Результат. Текст негативний.


Висновок: на лабораторній роботі я навчився створювати RNN для семантичного аналізу з використанням моделі GRU.